In [1]:
import pandas as pd
import duckdb

## Entrainement à la syntaxe

In [3]:
datapop = {
    "year": [2016, 2017, 2018, 2019, 2020] * 3,
    "region": (["IDF"] * 5) + (["HDF"] * 5) + (["PACA"] * 5),
    "population": 
    [1010000, 1020000, 1030000, 1040000, 1000000] + 
    [910000, 920000, 930000, 940000, 900000] + 
    [810000, 820000, 830000, 840000, 950000],
}

dfpop = pd.DataFrame(datapop)
dfpop

,year,region,population
0,2016,IDF,1010000
1,2017,IDF,1020000
2,2018,IDF,1030000
3,2019,IDF,1040000
4,2020,IDF,1000000
5,2016,HDF,910000
6,2017,HDF,920000
7,2018,HDF,930000
8,2019,HDF,940000
9,2020,HDF,900000


In [7]:
query = """
SELECT year, COALESCE(region, 'all regions') AS region, SUM(population) AS total_population
FROM dfpop
GROUP BY
GROUPING SETS ((year, region), year)
ORDER BY year, region
"""

duckdb.sql(query)

┌───────┬─────────────┬──────────────────┐
│ year  │   region    │ total_population │
│ int64 │   varchar   │      int128      │
├───────┼─────────────┼──────────────────┤
│  2016 │ HDF         │           910000 │
│  2016 │ IDF         │          1010000 │
│  2016 │ PACA        │           810000 │
│  2016 │ all regions │          2730000 │
│  2017 │ HDF         │           920000 │
│  2017 │ IDF         │          1020000 │
│  2017 │ PACA        │           820000 │
│  2017 │ all regions │          2760000 │
│  2018 │ HDF         │           930000 │
│  2018 │ IDF         │          1030000 │
│  2018 │ PACA        │           830000 │
│  2018 │ all regions │          2790000 │
│  2019 │ HDF         │           940000 │
│  2019 │ IDF         │          1040000 │
│  2019 │ PACA        │           840000 │
│  2019 │ all regions │          2820000 │
│  2020 │ HDF         │           900000 │
│  2020 │ IDF         │          1000000 │
│  2020 │ PACA        │           950000 │
│  2020 │ a

Groupez la population par:
- année, région
- année seulement

In [12]:
%load solutions/3regions_simple_grouping_sets.py

┌───────┬─────────┬─────────────────┐
│ year  │ region  │ sum(population) │
│ int64 │ varchar │     int128      │
├───────┼─────────┼─────────────────┤
│  2016 │ IDF     │         1010000 │
│  2017 │ IDF     │         1020000 │
│  2018 │ IDF     │         1030000 │
│  2019 │ IDF     │         1040000 │
│  2020 │ IDF     │         1000000 │
│  2016 │ HDF     │          910000 │
│  2017 │ HDF     │          920000 │
│  2018 │ HDF     │          930000 │
│  2019 │ HDF     │          940000 │
│  2020 │ HDF     │          900000 │
│  2016 │ PACA    │          810000 │
│  2017 │ PACA    │          820000 │
│  2018 │ PACA    │          830000 │
│  2019 │ PACA    │          840000 │
│  2020 │ PACA    │          950000 │
│  2016 │ NULL    │         2730000 │
│  2017 │ NULL    │         2760000 │
│  2018 │ NULL    │         2790000 │
│  2019 │ NULL    │         2820000 │
│  2020 │ NULL    │         2850000 │
├───────┴─────────┴─────────────────┤
│ 20 rows                 3 columns │
└───────────

Ici, le premier grouping set retourne toute la table, mais on pourrait imaginer une table qui contient les valeurs pour chaque départements, et il faudrait les rassembler dans leurs régions respectives

##### Data

Note: il est possible que vous n'arriviez pas à utiliser la commande pd.read_excel <br />
dans la cellule ci-dessous...

Si c'est le cas, utilisez:

!pip install xlrd >> /dev/null  (dans une cellule code, of course :) )

In [8]:
dpt_to_region_mapping = {
    "59":"HDF",
    "62":"HDF",
    "75":"IDF",
    "95":"IDF",
    "83":"PACA",
    "84":"PACA",
}

dpts_dfs = []
for year in range(2016,2022):
    # https://www.insee.fr/fr/statistiques/1893198
    df = pd.read_excel("data/estim-pop-dep-sexe-gca-1975-2023.xls", str(year), skiprows=3)
    subset = df[df["Départements"].isin(["59", "62", "75", "95", "83", "84"])][["Départements", "Unnamed: 7"]]
    subset["year"] = year
    dpts_dfs.append(subset)

dpts_dfs = pd.concat(dpts_dfs)
dpts_dfs.columns = ["departement", "population", "year"]
dpts_dfs["region"] = dpts_dfs["departement"].apply(lambda x: dpt_to_region_mapping.get(x))
dpts_dfs = dpts_dfs[["region", "departement", "year", "population"]]
dpts_dfs.head()

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.

## Grouping sets sur données de population réelle

Exercice: Obtenez le total de population en regroupant par:
- (year, region)
- year

In [14]:
%load solutions/4regions_real_data_grouping_sets.py

,region,year,sum(population)
0,HDF,2016,4074448.0
1,HDF,2017,4072379.0
2,HDF,2018,4072977.0
3,HDF,2019,4073624.0
4,HDF,2020,4069913.0
5,HDF,2021,4066673.0
6,IDF,2016,3412250.0
7,IDF,2017,3416144.0
8,IDF,2018,3414182.0
9,IDF,2019,3415097.0


On peut observer une baisse du nombre d'habitants en IDF entre 2019 et 2021, et une hausse en PACA.

Coïncidence ? ^^

Ici le regroupement par année ne nous sert pas, <br />
mais on pourrait s'en servir pour afficher  <br />
le pourcentage de la population totale <br />
que chaque région représente, année après année 